In [1]:
import numpy as np
import os
import PIL
import PIL.Image
import tensorflow as tf
import tensorflow_datasets as tfds

In [11]:
print(tf.__version__)

from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

2.5.0
[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 17744252773303032579
]


In [12]:
from keras_preprocessing.image import ImageDataGenerator

num_classes = 15

# create a data generator
datagen = ImageDataGenerator()
train_it = datagen.flow_from_directory('images/training', class_mode='binary', batch_size=64)
test_it = datagen.flow_from_directory('images/testing', batch_size=64)

Found 1500 images belonging to 15 classes.
Found 2985 images belonging to 1 classes.


In [14]:
#define model
model = tf.keras.Sequential([
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Conv2D(32, 3, activation='relu'),
    tf.keras.layers.MaxPooling2D(),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(num_classes)
])
model.compile(optimizer='adam',
              loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics=['accuracy'])

In [15]:
#train model
model.fit_generator(train_it, steps_per_epoch=24, epochs=10) #1500 / 64

Epoch 1/10
24/24 [==============================] - 13s 521ms/step - loss: 77.1345 - accuracy: 0.0767
Epoch 2/10
24/24 [==============================] - 13s 518ms/step - loss: 2.2499 - accuracy: 0.3133
Epoch 3/10
24/24 [==============================] - 12s 510ms/step - loss: 1.1811 - accuracy: 0.6553
Epoch 4/10
24/24 [==============================] - 12s 500ms/step - loss: 0.4427 - accuracy: 0.8900
Epoch 5/10
24/24 [==============================] - 13s 519ms/step - loss: 0.1981 - accuracy: 0.9513
Epoch 6/10
24/24 [==============================] - 13s 521ms/step - loss: 0.1636 - accuracy: 0.9633
Epoch 7/10
24/24 [==============================] - 12s 507ms/step - loss: 0.0933 - accuracy: 0.9813
Epoch 8/10
24/24 [==============================] - 12s 508ms/step - loss: 0.0628 - accuracy: 0.9873
Epoch 9/10
24/24 [==============================] - 13s 522ms/step - loss: 0.0571 - accuracy: 0.9907
Epoch 10/10
24/24 [==============================] - 12s 504ms/step - loss: 0.0882 - accur

In [16]:
# evaluate model

class_labels = ["bedroom", "Coast", "Forest", "Highway", "industrial", "Insidecity", "kitchen", "livingroom", "Mountain", "Office", "OpenCountry", "store", "Street", "Suburb", "TallBuilding"]

df = model.predict_classes(test_it, batch_size=64)
with open('run3.txt', 'w') as f:
    for row in df:
        f.write(str(row)+".jpg "+class_labels[row]+"\n")

C:\Users\jakub\anaconda3\envs\py39\lib\site-packages\tensorflow\python\keras\engine\sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '
